# LLM-Based Exercise Recommendation System
## Two-LLM Architecture for Physiotherapy Exercise Prescription

**System Flow:**
1. Fetch patient data from Supabase (questionnaire, STS assessment, demographics)
2. LLM #1: Exercise Recommendation Agent → Select 4 exercises based on clinical goals
3. LLM #2: Safety Verification Agent → Review and approve/modify/reject for safety
4. Display final prescription with clinical rationale

---

## Cell 1: Setup and Initialization

In [1]:
import os
from dotenv import load_dotenv
from langchain_deepseek import ChatDeepSeek
from supabase import create_client
import json

# Load environment variables
load_dotenv()

# Get API keys
deepseek_api_key = os.environ.get("DEEPSEEK_API_KEY")
supabase_url = os.environ.get("VITE_SUPABASE_URL")
supabase_key = os.environ.get("VITE_SUPABASE_ANON_KEY")

# Initialize LLM
llm = ChatDeepSeek(
    model="deepseek-chat",
    api_key=deepseek_api_key,
    temperature=0  # Deterministic for clinical consistency
)

# Initialize Supabase client
supabase = create_client(supabase_url, supabase_key)

print("✓ LLM initialized (DeepSeek Chat)")
print("✓ Supabase client initialized")
print("\nReady to process patient data!")

✓ LLM initialized (DeepSeek Chat)
✓ Supabase client initialized

Ready to process patient data!


## Cell 2: Import Custom Modules

In [2]:
# Import data fetching module
from data_fetcher import (
    fetch_patient_data,
    fetch_all_exercises,
    structure_patient_profile,
    print_patient_summary
)

# Import LLM agents
from llm1_recommendation import (
    generate_exercise_recommendations,
    print_llm1_output
)
from llm2_safety_verification import (
    verify_safety_and_finalize,
    print_llm2_output
)

print("✓ All modules imported successfully")

✓ All modules imported successfully


## Cell 3: User Input - Select Patient

In [3]:
# Enter username from database
# Test team should use this after filling out the HTML questionnaire
username = input("Enter patient username from database: ")

print(f"\n📋 Fetching data for patient: {username}")
print("="*80)

Enter patient username from database: user01

📋 Fetching data for patient: user01


## Cell 4: Fetch and Structure Patient Data

In [4]:
try:
    # Fetch raw data from Supabase
    print("Fetching patient data from Supabase...")
    raw_data = fetch_patient_data(supabase, username)
    
    # Fetch exercise database
    print("Fetching exercise database...")
    exercises = fetch_all_exercises(supabase)
    print(f"Loaded {len(exercises)} exercises")
    
    # Structure data for LLM consumption
    print("\nStructuring patient profile...")
    patient_profile = structure_patient_profile(raw_data, exercises)
    
    # Print summary
    print_patient_summary(patient_profile)
    
    print("\n✓ Data fetching complete!")
    
except ValueError as e:
    print(f"❌ ERROR: {e}")
    print("\nPlease ensure:")
    print("1. The username exists in the database")
    print("2. Patient has completed questionnaire")
    print("3. Patient has completed STS assessment")
    print("4. Patient demographics are recorded")
except Exception as e:
    print(f"❌ Unexpected error: {e}")
    raise

Fetching patient data from Supabase...
Fetching exercise database...
Loaded 32 exercises

Structuring patient profile...
PATIENT PROFILE SUMMARY

Demographics:
  Age: 66 | Gender: female
  Height: 160.0 cm | Weight: 80.0 kg

STS Assessment:
  Repetitions: 9
  HK Norm (Average): 11 - 14
  Performance: Below Average
  Knee alignment: valgus
  Trunk sway: absent | Hip sway: present

Questionnaire Sections (normalized 0-100, higher=better):
  symptoms            :  60.0 (avg: 2.20)
  stiffness           :  66.7 (avg: 2.00)
  pain                :  77.8 (avg: 1.67)
  function_ADL        :  88.2 (avg: 1.35)
  function_sports     :  46.7 (avg: 2.60)
  quality_of_life     :  58.3 (avg: 2.25)

Position-Relevant Questions (0=None, 1=Mild, 2=Moderate, 3=Severe, 4=Extreme):
  Weight-bearing:
    F1: Descending stairs = 4
    F2: Ascending stairs = 3
    F4: Standing = 0
    SP1: Squatting = 2
    SP4: Twisting/pivoting on your injured knee = 2
  Quadruped:
    SP5: Kneeling = 1
  Lying: Safe by de

In [5]:
patient_profile

{'demographics': {'age': 66,
  'gender': 'female',
  'height_cm': 160.0,
  'weight_kg': 80.0,
  'date_of_birth': '1960-01-01'},
 'questionnaire_sections': {'symptoms': {'questions': ['s1',
    's2',
    's3',
    's4',
    's5'],
   'scores': [3, 2, 2, 1, 3],
   'avg': 2.2,
   'total': 11,
   'normalized_0_100': 60.0},
  'stiffness': {'questions': ['st1', 'st2'],
   'scores': [2, 2],
   'avg': 2.0,
   'total': 4,
   'normalized_0_100': 66.7},
  'pain': {'questions': ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9'],
   'scores': [3, 2, 1, 1, 2, 3, 0, 1, 2],
   'avg': 1.67,
   'total': 15,
   'normalized_0_100': 77.8},
  'function_ADL': {'questions': ['f1',
    'f2',
    'f3',
    'f4',
    'f5',
    'f6',
    'f7',
    'f8',
    'f9',
    'f10',
    'f11',
    'f12',
    'f13',
    'f14',
    'f15',
    'f16',
    'f17'],
   'scores': [4, 3, 1, 0, 2, 1, 1, 1, 2, 1, 2, 0, 1, 0, 1, 2, 1],
   'avg': 1.35,
   'total': 23,
   'normalized_0_100': 88.2},
  'function_sports': {'questions'

## Cell 5: Run LLM #1 - Exercise Recommendation Agent

LLM #1 analyzes patient capability and biomechanical needs, then recommends 4 exercises.

In [ ]:
print("\n" + "="*80)
print("Running LLM #1: Exercise Recommendation Agent")
print("="*80 + "\n")
print("Analyzing patient and selecting exercises...\n")

try:
    # Generate recommendations
    llm1_output = generate_exercise_recommendations(llm, patient_profile)
    
    # Print formatted output
    print_llm1_output(llm1_output)
    
    print("\n✓ LLM #1 recommendations complete!")
    
except Exception as e:
    print(f"❌ ERROR in LLM #1: {e}")
    raise

## Cell 6: Run LLM #2 - Safety Verification Agent

LLM #2 reviews proposed exercises and verifies safety against objective clinical measures.

In [ ]:
print("\n" + "="*80)
print("Running LLM #2: Safety Verification Agent")
print("="*80 + "\n")
print("Verifying safety of proposed exercises...\n")

try:
    # Verify safety and finalize
    llm2_output = verify_safety_and_finalize(llm, patient_profile, llm1_output)
    
    # Print formatted output
    print_llm2_output(llm2_output)
    
    print("\n✓ LLM #2 safety verification complete!")
    
except Exception as e:
    print(f"❌ ERROR in LLM #2: {e}")
    raise

## Cell 7: Display Final Prescription

Final exercise prescription after safety review and modifications.

In [ ]:
print("\n" + "#"*80)
print("#" + " "*78 + "#")
print("#" + " "*24 + "FINAL EXERCISE PRESCRIPTION" + " "*28 + "#")
print("#" + " "*78 + "#")
print("#"*80 + "\n")

print(f"Patient: {username}")
print(f"Age: {patient_profile['demographics']['age']} | Gender: {patient_profile['demographics']['gender']}")
print(f"STS Performance: {patient_profile['sts_assessment']['repetition_count']} reps "
      f"({patient_profile['sts_assessment']['benchmark_percent']}% of benchmark)")
print(f"Biomechanical Issues: {patient_profile['sts_assessment']['knee_alignment']} knee, "
      f"Flexibility: {patient_profile['flexibility']['toe_touch_test']} touch toes")
print("\n" + "-"*80 + "\n")

for i, ex in enumerate(llm2_output['final_prescription'], 1):
    print(f"Exercise {i}: {ex['exercise_name']}")
    print(f"           {ex['exercise_name_ch']}")
    print(f"")
    # Handle positions array (new schema v3.0)
    positions_display = ', '.join([p.replace('_', ' ').title() for p in ex['positions']])
    print(f"  Positions: {positions_display}")
    print(f"  Difficulty: {ex['difficulty']}/10")
    print(f"")
    print(f"  Clinical Rationale:")
    print(f"  {ex['clinical_rationale']}")
    
    if ex['modifications']:
        print(f"")
        print(f"  ⚠️  Safety Modifications:")
        for j, mod in enumerate(ex['modifications'], 1):
            print(f"      {j}. {mod}")
    
    print("\n" + "-"*80 + "\n")

print("\n✅ Exercise prescription complete!")
print("\nRecommendation: Review with supervising physiotherapist before patient execution.")

## Cell 8 (Optional): Export Results to JSON

In [ ]:
# Export complete results for record-keeping
output_data = {
    'patient_username': username,
    'patient_profile_summary': {
        'age': patient_profile['demographics']['age'],
        'gender': patient_profile['demographics']['gender'],
        'sts_benchmark_percent': patient_profile['sts_assessment']['benchmark_percent'],
        'knee_alignment': patient_profile['sts_assessment']['knee_alignment'],
        'flexibility': patient_profile['flexibility']['toe_touch_test']
    },
    'llm1_recommendations': llm1_output,
    'llm2_safety_review': llm2_output
}

# Save to file
output_filename = f"prescription_{username}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(output_filename, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"✓ Results exported to: {output_filename}")

## Cell 9 (Optional): Compare with Rule-Based Algorithm

Compare LLM recommendations with the original rule-based algorithm.

In [ ]:
# TODO: Implement comparison with rule-based algorithm from algorithm.js
# This would require porting the JavaScript algorithm to Python or calling it via Node

print("Comparison with rule-based algorithm:")
print("[To be implemented - requires porting algorithm.js to Python]")
print("\nManual comparison points:")
print("1. Position selection: Does LLM choose similar positions as rule-based?")
print("2. Difficulty matching: Are difficulty levels appropriate?")
print("3. Biomechanical targeting: Does LLM correctly identify valgus/varus/flexibility needs?")
print("4. Safety: Are constraints properly enforced?")